In [3]:
## Authors: Raja Batra And Eli Rejto
## October 19, 2023

hi


In [6]:
pip install torchtext


Note: you may need to restart the kernel to use updated packages.


In [8]:
conda install torchtect

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - torchtect

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [10]:
import torch
import numpy as np

In [13]:
spanish_file_path = 'dataforbuildingmodel/somespanishtext.txt'
with open(spanish_file_path, 'r', encoding='latin-1') as file:
    spanish_text = file.read().lower()  # Convert to lowercase
    spanish_text = ''.join(char if char.isalpha() or char.isspace() or char == '.' else ' ' for char in spanish_text)

In [1]:
print(spanish_text[0:100])

NameError: name 'spanish_text' is not defined

In [ ]:
english_file_path = 'data/wikitext-103/wiki.train.tokens'
with open(english_file_path, 'r', encoding='latin-1') as file:
    english_text = file.read().lower()  # Convert to lowercase
    english_text = ''.join(char if char.isalpha() or char.isspace() or char == '.' else ' ' for char in english_text)